<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
  Using cached wget-3.2.zip (10 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=6f6e95ccc7c7a0464cd2e53ead844ef12be314f69cfa12a7dad59fd44cfdd7a4
  Stored in directory: /Users/camillefarrell/Library/Caches/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [8]:
import folium
import wget
import ssl
import pandas as pd

In [13]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
import ssl
import wget
import urllib.request

# Disable SSL verification globally
ssl._create_default_https_context = ssl._create_unverified_context


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [14]:
#Work around for SSL certificate verification issues
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
ssl_context = ssl._create_unverified_context()

response = urllib.request.urlopen(url, context=ssl_context)
spacex_df = pd.read_csv(response)
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [17]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [33]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [34]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [20]:
# Iterate through each launch site in the dataframe
for index, site in launch_sites_df.iterrows():
    # Extract the latitude and longitude
    coordinate = [site['Lat'], site['Long']]
    # Add a circle for the launch site
    circle = folium.Circle(
        coordinate,
        radius=1000,
        color='#d35400',
        fill=True,
        fill_color='#d35400'
    ).add_child(folium.Popup(site['Launch Site']))
    # Add a marker for the launch site
    marker = folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
        )
    )
    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
Based on the map and the coordinates of the launch sites:

1. **Are all launch sites in proximity to the Equator line?**
    - No, not all launch sites are in close proximity to the Equator. While the launch sites in Florida (CCAFS LC-40, CCAFS SLC-40, and KSC LC-39A) are relatively closer to the Equator (latitude ~28°N), the VAFB SLC-4E site in California is farther north (latitude ~34°N). Proximity to the Equator is advantageous for launches to geostationary orbits due to the Earth's rotational velocity, but not all launch sites are located near the Equator.

2. **Are all launch sites in very close proximity to the coast?**
    - Yes, all launch sites are in close proximity to the coast. This is because launching rockets over water reduces the risk to populated areas in case of launch failures or falling debris. Coastal locations also provide a clear trajectory for rockets to reach orbit without overflying land.

**Findings:**
- Launch sites are strategically located to balance safety, efficiency, and mission requirements. Coastal locations minimize risks, while proximity to the Equator is beneficial but not mandatory, depending on the type of orbit being targeted.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [35]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [36]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [37]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to the 'class' column to create a new 'marker_color' column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [38]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Iterate through each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Extract the latitude, longitude, and marker color
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    
    # Create a Marker object with its coordinate and customize the icon color
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color)
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the updated map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add the MousePosition plugin to the map
site_map.add_child(mouse_position)

# Save the map to an HTML file
site_map.save("site_map.html")

# Open the map in the default web browser
import webbrowser
webbrowser.open("site_map.html")

site_map

True

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [39]:
from folium.plugins import MousePosition

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add the MousePosition plugin to the map
site_map.add_child(mouse_position)

# Save the map to an HTML file
site_map.save("site_map.html")

# Open the map in the default web browser
import webbrowser
webbrowser.open("site_map.html")

site_map

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [40]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [49]:
lat1 = 28.5721  # Example latitude of Launch Site 1
lon1 = -80.6480  # Example longitude of Launch Site 1
lat2 = 28.56349  # Corrected latitude for the nearest coastline
lon2 = -80.56797  # Corrected longitude for the nearest coastline

distance = calculate_distance(lat1, lon1, lat2, lon2)
print(distance)
print(f"Distance between Launch Site 1 and the closest coastline: {distance:.2f} km")

7.87639484830441
Distance between Launch Site 1 and the closest coastline: 7.88 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [50]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
distance_marker = folium.Marker(
    [lat2, lon2],  # Coordinates of the closest coastline point
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Display the updated map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [51]:
# Define the coordinates for the PolyLine
coordinates = [coordinate, [lat2, lon2]]

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)

# Display the updated map
site_map

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [53]:
# Coordinates for the closest city, railway, and highway
city_coordinate = [28.612, -80.807]  # Example coordinates for the closest city
railway_coordinate = [28.572, -80.585]  # Example coordinates for the closest railway
highway_coordinate = [28.563, -80.570]  # Example coordinates for the closest highway

# Calculate distances to the launch site
city_distance = calculate_distance(coordinate[0], coordinate[1], city_coordinate[0], city_coordinate[1])
railway_distance = calculate_distance(coordinate[0], coordinate[1], railway_coordinate[0], railway_coordinate[1])
highway_distance = calculate_distance(coordinate[0], coordinate[1], highway_coordinate[0], highway_coordinate[1])

# Create markers for the closest city, railway, and highway
city_marker = folium.Marker(
    city_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(city_distance),
    )
)
railway_marker = folium.Marker(
    railway_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(railway_distance),
    )
)
highway_marker = folium.Marker(
    highway_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(highway_distance),
    )
)

# Add the markers to the map
site_map.add_child(city_marker)
site_map.add_child(railway_marker)
site_map.add_child(highway_marker)

# Draw lines between the launch site and the closest city, railway, and highway
city_line = folium.PolyLine(locations=[coordinate, city_coordinate], weight=1, color='blue')
railway_line = folium.PolyLine(locations=[coordinate, railway_coordinate], weight=1, color='green')
highway_line = folium.PolyLine(locations=[coordinate, highway_coordinate], weight=1, color='red')

# Add the lines to the map
site_map.add_child(city_line)
site_map.add_child(railway_line)
site_map.add_child(highway_line)

# Display the updated map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [ ]:
### Findings:

1. **Are launch sites in close proximity to railways?**
    - Yes, launch sites are in close proximity to railways. For example, the distance between the CCAFS SLC-40 launch site and the nearest railway is approximately 1.26 km. This proximity facilitates the transportation of heavy equipment and materials required for launches.

2. **Are launch sites in close proximity to highways?**
    - Yes, launch sites are also close to highways. For instance, the distance between the CCAFS SLC-40 launch site and the nearest highway is approximately 0.67 km. Highways provide easy access for personnel and logistics.

3. **Are launch sites in close proximity to coastline?**
    - Yes, all launch sites are located very close to the coastline. For example, the distance between the CCAFS SLC-40 launch site and the nearest coastline is approximately 7.88 km. Coastal locations are chosen to ensure that rockets can launch over water, minimizing risks to populated areas in case of failures.

4. **Do launch sites keep certain distance away from cities?**
    - Yes, launch sites are generally located at a safe distance from cities. For example, the distance between the CCAFS SLC-40 launch site and the nearest city is approximately 23.13 km. This ensures safety for urban populations and reduces noise and environmental impact.

### Explanation:
Launch sites are strategically located to balance safety, accessibility, and operational efficiency. Proximity to railways and highways ensures smooth transportation of materials and personnel. Being close to the coastline minimizes risks during launches, while maintaining a safe distance from cities ensures public safety and reduces disturbances.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
